# Gradient Vanishing problem

<img src='https://i.imgur.com/fzwNUpF.png' width='100%'>

# ReLU

* sigmoid 함수에 비해 계산량이 작다.
* ReLU의 음숫값이 0으로 죽는 현상을 개선하여 ReakyReLU 개발

In [0]:
# XOR_DNN_TF2.ipynb

import tensorflow as tf
import numpy as np

In [0]:
x = np.array([[0,0],[0,1],[1,0],[1,1]]).astype('float32')
y = np.array([[0],[1],[1],[0]]).astype('float32')
x, y

(array([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], dtype=float32), array([[0.],
        [1.],
        [1.],
        [0.]], dtype=float32))

In [0]:
from tensorflow.keras import layers

#actFunc = 'sigmoid' # 학습이 안된다.
actFunc = 'relu' # 된다.
model = tf.keras.Sequential()
model.add(layers.Dense(10, activation=actFunc, input_dim=2))
model.add(layers.Dense(10, activation=actFunc))
model.add(layers.Dense(10, activation=actFunc))
model.add(layers.Dense(10, activation=actFunc))
model.add(layers.Dense(10, activation=actFunc))
model.add(layers.Dense(10, activation=actFunc))
model.add(layers.Dense(10, activation=actFunc))
model.add(layers.Dense(1, activation='sigmoid'))    # 출력값을 0~1로 만들기 위함
sgd = tf.keras.optimizers.SGD(learning_rate=0.1)
model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
model.fit(x, y, epochs=10000, batch_size=4, verbose=0)
model.evaluate(x, y)

1/1 [==============================] - 0s 1ms/step - loss: 7.1104e-06 - accuracy: 1.0000


[7.110358637874015e-06, 1.0]

In [0]:
predicted = model.predict(x)
predicted

array([[2.0134628e-05],
       [9.9999547e-01],
       [9.9999881e-01],
       [2.5657196e-06]], dtype=float32)

# Initialization

신경망 모형에서 초기값에 따라 결과가 달라지기도 하고 해에 수렴하는 속도도 달라진다.<br>
그런 이유로 초기값을 어떻게 줄 것이냐 역시 연구의 대상이다.<br>
가중치의 분산을 일정 수준 이하로 작게 만들면 더 빠른 수렴을 보인다는 연구결과에 의해 나타난 방법이 Xavier 초기값이다.

W=np.random.randn(fan_in, fan_out)/np.sqrt(fan_in) # Xavier initialization(2010)<br>
W=np.random.randn(fan_in, fan_out)/np.sqrt(fan_in/2) #He initialization(2015)
<br><br>
np.random.randn(): 표준정규분포의 난수<br>
fan_in: 이전 뉴런의 수<br>
fan_out: 이후 뉴런의 수

# Dropout

결과에 많은 영향을 미쳤던 노드 뿐만 아니라 다른 노드들도 학습을 해주는 방법<br><br>

tf.keras.layers.Dropout(0.3): 30%를 죽인다

# Batch Normalization

<img src='https://i.imgur.com/3yUB7CG.png' width='100%'>

**Important**<br>
* 배치 정규화는 미니 배치 처리에서 배치 마다 조금씩 분포가 달라지는 문제를 해결하기 위해 배치단위로 정규화를 실시하는 아이디어다.
* 학습시, 배치 단위로 정규화하고, Test 시에는 학습 데이터의 평균과 분산을 이용하여 위의 식으로 변환한 후 Test를 진행하면 된다.

# Optimizer

MNIST dataset을 예로 들어보자. MNIST dataset은 총 60000개의 이미지가 존재하고, 각 이미지는 (28,28)의 크기를 갖는다. 당연히 전체 dataset을 학습시키면 각 epoch마다 도출되는 cost값은 점점 줄어드는 추세이다. 그런데 이 방법은 각 epoch을 계산하는 시간이 오래 걸리기 때문에 전체 dataset을 batch로 나눠 batch 단위로 학습하고 cost가 도출된다. 우리는 이러한 방법을 SGD(Stochastic Gradient Descent)라고 한다. 왜냐하면 batch마다 도출되는 cost가 계속해서 줄어들지 않을 수 있기 때문이다. 이는 전체 dataset은 표준화되어있지만, 각 batch로 봤을 땐 표준화되어있지 않기 때문이다. 결국 cost curve는 전체적으로 감소하는 그래프이지만 들쭉날쭉하며 불안전한 양상을 보일것이다.

In [0]:
# Mnist_DNN_TF2.ipynb

import tensorflow as tf
import numpy as np

In [0]:
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
mnist = datasets.mnist
(train_x, train_y), (test_x, test_y) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
train_x.shape

(60000, 28, 28)

In [0]:
train_x = train_x.reshape(-1,784) 
test_x = test_x.reshape(-1,784) 

In [0]:
train_x = train_x / 255
test_x = test_x / 255

In [0]:
train_y_onehot = to_categorical(train_y)
test_y_onehot = to_categorical(test_y)
train_y_onehot[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [0]:
#Hidden Layer 추가
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=784))  # weights: 784x256+256
model.add(layers.Dense(256, activation='relu'))                 # weights: 256x256+256
model.add(layers.Dense(10, activation='softmax'))               # weights: 256x10+10
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_x, train_y_onehot, batch_size = 100, epochs=5)

Epoch 1/5
600/600 [==============================] - 1s 2ms/step - loss: 1.0676 - accuracy: 0.7455
Epoch 2/5
600/600 [==============================] - 1s 2ms/step - loss: 0.4301 - accuracy: 0.8845
Epoch 3/5
600/600 [==============================] - 1s 2ms/step - loss: 0.3470 - accuracy: 0.9022
Epoch 4/5
600/600 [==============================] - 1s 2ms/step - loss: 0.3089 - accuracy: 0.9122
Epoch 5/5
600/600 [==============================] - 1s 2ms/step - loss: 0.2837 - accuracy: 0.9191


In [0]:
# adam 옵티마이저 사용
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=784))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_x, train_y_onehot, batch_size = 100, epochs=5)

Epoch 1/5
600/600 [==============================] - 1s 2ms/step - loss: 0.2425 - accuracy: 0.9290
Epoch 2/5
600/600 [==============================] - 1s 2ms/step - loss: 0.0914 - accuracy: 0.9722
Epoch 3/5
600/600 [==============================] - 1s 2ms/step - loss: 0.0602 - accuracy: 0.9815
Epoch 4/5
600/600 [==============================] - 1s 2ms/step - loss: 0.0427 - accuracy: 0.9866
Epoch 5/5
600/600 [==============================] - 1s 2ms/step - loss: 0.0322 - accuracy: 0.9898


In [0]:
# adam 옵티마이저 + He init.
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=784, kernel_initializer='he_normal'))
model.add(layers.Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(layers.Dense(10, activation='softmax', kernel_initializer='he_normal'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_x, train_y_onehot, batch_size = 100, epochs=5)

Epoch 1/5
600/600 [==============================] - 1s 2ms/step - loss: 0.2361 - accuracy: 0.9293
Epoch 2/5
600/600 [==============================] - 1s 2ms/step - loss: 0.0898 - accuracy: 0.9730
Epoch 3/5
600/600 [==============================] - 1s 2ms/step - loss: 0.0599 - accuracy: 0.9809
Epoch 4/5
600/600 [==============================] - 1s 2ms/step - loss: 0.0414 - accuracy: 0.9869
Epoch 5/5
600/600 [==============================] - 1s 2ms/step - loss: 0.0316 - accuracy: 0.9899


In [0]:
model.evaluate(test_x, test_y_onehot)

313/313 [==============================] - 1s 2ms/step - loss: 0.0710 - accuracy: 0.9778


[0.07101818174123764, 0.9778000116348267]

In [0]:
# adam optimizer와 He 초기화 방법, 드롭아웃 추가
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(layers.Dense(256, activation='relu', kernel_initializer='he_normal', input_dim=784))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.fit(train_x, train_y_onehot, batch_size = 100, epochs=10)

Epoch 1/10
600/600 [==============================] - 1s 2ms/step - loss: 0.3351 - accuracy: 0.8977
Epoch 2/10
600/600 [==============================] - 1s 2ms/step - loss: 0.1500 - accuracy: 0.9544
Epoch 3/10
600/600 [==============================] - 1s 2ms/step - loss: 0.1102 - accuracy: 0.9665
Epoch 4/10
600/600 [==============================] - 1s 2ms/step - loss: 0.0925 - accuracy: 0.9712
Epoch 5/10
600/600 [==============================] - 1s 2ms/step - loss: 0.0791 - accuracy: 0.9756
Epoch 6/10
600/600 [==============================] - 1s 2ms/step - loss: 0.0709 - accuracy: 0.9773
Epoch 7/10
600/600 [==============================] - 1s 2ms/step - loss: 0.0613 - accuracy: 0.9811
Epoch 8/10
600/600 [==============================] - 1s 2ms/step - loss: 0.0590 - accuracy: 0.9808
Epoch 9/10
600/600 [==============================] - 1s 2ms/step - loss: 0.0522 - accuracy: 0.9832
Epoch 10/10
600/600 [==============================] - 1s 2ms/step - loss: 0.0479 - accuracy: 0.9848

In [0]:
model.evaluate(test_x, test_y_onehot)

313/313 [==============================] - 1s 2ms/step - loss: 0.0613 - accuracy: 0.9828


[0.06131801754236221, 0.9828000068664551]

In [0]:
# 오버피팅 상황 만들기
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=784))

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_x, train_y_onehot, batch_size = 100, epochs=5)

Epoch 1/5
600/600 [==============================] - 2s 3ms/step - loss: 0.2729 - accuracy: 0.9184
Epoch 2/5
600/600 [==============================] - 2s 3ms/step - loss: 0.1187 - accuracy: 0.9691
Epoch 3/5
600/600 [==============================] - 2s 3ms/step - loss: 0.0939 - accuracy: 0.9755
Epoch 4/5
600/600 [==============================] - 2s 3ms/step - loss: 0.0771 - accuracy: 0.9800
Epoch 5/5
600/600 [==============================] - 2s 3ms/step - loss: 0.0667 - accuracy: 0.9836


In [0]:
model.evaluate(test_x, test_y_onehot)

313/313 [==============================] - 1s 2ms/step - loss: 0.0903 - accuracy: 0.9779


[0.09032953530550003, 0.9779000282287598]

In [0]:
model.save("test.h5")

In [0]:
!ls 

sample_data  test.h5


In [0]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
#from tensorflow import keras
mnist = datasets.mnist
(train_x, train_y), (test_x, test_y) = mnist.load_data()
train_x = train_x.reshape(-1,784) 
test_x = test_x.reshape(-1,784)
train_x = train_x / 255
test_x = test_x / 255
train_y_onehot = to_categorical(train_y)
test_y_onehot = to_categorical(test_y)
train_y_onehot[0]

model = tf.keras.models.load_model('test.h5')
model.evaluate(test_x, test_y_onehot)

313/313 [==============================] - 1s 2ms/step - loss: 0.0903 - accuracy: 0.9779


[0.09032953530550003, 0.9779000282287598]

In [0]:
import tensorflow as tf
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')

RuntimeError: ignored

In [0]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)
c

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[22., 28.],
       [49., 64.]], dtype=float32)>

# Quiz

1. dropout 설명에서 나온 축구선수가 아닌 것은?
 * 박지성(O), 손흥민(O), 안정환
2. SGD에서 S는 무엇의 약자인가?
 * Stochastic

3. 미니배치에서 6만개의 mnist 이미지를 배치크기 50개로 하면 미지수 갱신 회수는 몇회인가?
 * 60000 / 50 = 1,200

4. mnist 데이터에서 이미지 크기는?
 * 28 * 28